### Inférence préparation

In [2]:
import cv2
import torch
import torchvision
import time
from torch2trt import TRTModule
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera
from utils import preprocess
import numpy as np

# Categories and device
CATEGORIES = ['Train']
device = torch.device('cuda')

# Model initialization
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

# Load both TRT models
model_trt_1 = TRTModule()
model_trt_1.load_state_dict(torch.load('RF1.pth'))

model_trt_2 = TRTModule()
model_trt_2.load_state_dict(torch.load('VD_trt.pth'))

# Initialize the car and camera
car = NvidiaRacecar()
camera = CSICamera(width=224, height=224, capture_fps=65)

FileNotFoundError: [Errno 2] No such file or directory: 'O2trt.pth'

### Correcteur Proportionnel

In [ ]:
# Steering parameters
STEERING_GAIN = -4.00
STEERING_BIAS = 0

# Set initial throttle
car.throttle = -0.3
frame_number = 0

# Start time
start_time = time.time()

# Main loop
while True:
    a = time.time()
    # Get the current time
    current_time = time.time()
    
    # Calculate elapsed time
    elapsed_time = current_time - start_time
    
    # Switch model every 60 seconds
    if int(elapsed_time) % 120 < 60:
        model_trt = model_trt_1
    else:
        model_trt = model_trt_2
    
    # Read image from camera
    image = camera.read()
    
    # Preprocess image
    image = preprocess(image).half()
    
    # Predict steering
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    
    # Apply steering
    car.steering = x * STEERING_GAIN + STEERING_BIAS

### Correcteur Proportionel derivateur

In [ ]:
# Steering parameters
kp = 2.5
kd = 8.0
last_x = 0

# Set initial throttle
car.throttle = -0.3
frame_number = 0

# Start time
start_time = time.time()

# Main loop
while True:
    a = time.time()
    # Get the current time
    current_time = time.time()
    
    # Calculate elapsed time
    elapsed_time = current_time - start_time
    
    # Switch model every 60 seconds
    if int(elapsed_time) % 120 < 60:
        model_trt = model_trt_1
    else:
        model_trt = model_trt_2
    
    # Read image from camera
    image = camera.read()
    
    # Preprocess image
    image = preprocess(image).half()
    
    # Predict steering
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    
    # Apply steering
    car.steering = - (x * kp + ( x - last_x ) *kd)
    last_x = x